# Polar coordinates with jaxfun

In [1]:
import numpy as np
import sympy as sp

from jaxfun import *
from jaxfun.Basespace import Domain

# system = 'polar'
system = "cylindrical"
r, theta, z = sp.symbols("r,theta,z", real=True, positive=True)

if system == "polar":
    C = get_CoordSys("C", sp.Lambda((r, theta), (r * sp.cos(theta), r * sp.sin(theta))))
    R = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0.5, 1),
        name="R",
        fun_str="phi",
    )
    T = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0, np.pi),
        name="T",
        fun_str="psi",
    )
    P = TensorProductSpace((R, T), system=C, name="P")

elif system == "cylindrical":
    C = get_CoordSys(
        "C", sp.Lambda((r, theta, z), (r * sp.cos(theta), r * sp.sin(theta), z))
    )
    R = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0.5, 1),
        name="R",
        fun_str="phi",
    )
    T = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0, np.pi),
        name="T",
        fun_str="psi",
    )
    Z = FunctionSpace(20, Legendre.Legendre, domain=Domain(0, 1), name="Z", fun_str="L")
    P = TensorProductSpace((R, T, Z), system=C, name="P")

In [2]:
u = TrialFunction(P, name="u")
du = Div(Grad(u))
du

Div(Grad(u(x, y, z; P)))

In [3]:
du.doit()

phi_l(r)*psi_m(theta)*Derivative(L_n(z), (z, 2)) + (r*psi_m(theta)*L_n(z)*Derivative(phi_l(r), (r, 2)) + psi_m(theta)*L_n(z)*Derivative(phi_l(r), r))/r + phi_l(r)*L_n(z)*Derivative(psi_m(theta), (theta, 2))/r**2

In [4]:
from sympy import srepr

srepr(du)

"Div(Grad(TrialFunction(x, y, z, Symbol('P'))))"

In [5]:
du = (C.x + C.y + C.z) * Div(Grad(u))
du

(x + y + z)*Div(Grad(u(x, y, z; P)))

In [6]:
srepr(du)

"Mul(Add(x, y, z), Div(Grad(TrialFunction(x, y, z, Symbol('P')))))"

In [7]:
du.doit()

(r*sin(theta) + r*cos(theta) + z)*(phi_l(r)*psi_m(theta)*Derivative(L_n(z), (z, 2)) + (r*psi_m(theta)*L_n(z)*Derivative(phi_l(r), (r, 2)) + psi_m(theta)*L_n(z)*Derivative(phi_l(r), r))/r + phi_l(r)*L_n(z)*Derivative(psi_m(theta), (theta, 2))/r**2)

In [8]:
Grad(u).doit()

(psi_m(theta)*L_n(z)*Derivative(phi_l(r), r))*C.b_r + (phi_l(r)*L_n(z)*Derivative(psi_m(theta), theta)/r**2)*C.b_theta + (phi_l(r)*psi_m(theta)*Derivative(L_n(z), z))*C.b_z

In [9]:
g = arguments.ScalarFunction("g", C)
g

g(x, y, z)

In [10]:
G = g.doit()
G

G(r, theta, z)

In [11]:
Div(Grad(g))

Div(Grad(g(x, y, z)))

In [12]:
Div(Grad(g)).doit()

Derivative(G(r, theta, z), (z, 2)) + (r*Derivative(G(r, theta, z), (r, 2)) + Derivative(G(r, theta, z), r))/r + Derivative(G(r, theta, z), (theta, 2))/r**2

In [13]:
P.tensorname

'R⊗T⊗Z'

In [14]:
V = VectorTensorProductSpace(P, name="V")
V.tensorname

'P×P×P'

In [15]:
V.name

'V'

In [16]:
v = TestFunction(V, name="v")
v

v(x, y, z; V)

In [17]:
v.doit()

(phi_i^{(0)}(r)*psi_j^{(0)}(theta)*L_k^{(0)}(z))*C.b_r + (phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z))*C.b_theta + (phi_i^{(2)}(r)*psi_j^{(2)}(theta)*L_k^{(2)}(z))*C.b_z

In [18]:
Cross(v, v).doit()

0

In [19]:
c = Cross(v, C.b_r)
c

Cross(v(x, y, z; V), C.b_r)

In [20]:
c.doit()

(phi_i^{(2)}(r)*psi_j^{(2)}(theta)*L_k^{(2)}(z)/r)*C.b_theta + (-r*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z))*C.b_z

In [21]:
d = Curl(v)
d

Curl(v(x, y, z; V))

In [22]:
d.doit()

((-r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*Derivative(L_k^{(1)}(z), z) + phi_i^{(2)}(r)*L_k^{(2)}(z)*Derivative(psi_j^{(2)}(theta), theta))/r)*C.b_r + ((phi_i^{(0)}(r)*psi_j^{(0)}(theta)*Derivative(L_k^{(0)}(z), z) - psi_j^{(2)}(theta)*L_k^{(2)}(z)*Derivative(phi_i^{(2)}(r), r))/r)*C.b_theta + ((r**2*psi_j^{(1)}(theta)*L_k^{(1)}(z)*Derivative(phi_i^{(1)}(r), r) + 2*r*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z) - phi_i^{(0)}(r)*L_k^{(0)}(z)*Derivative(psi_j^{(0)}(theta), theta))/r)*C.b_z

In [23]:
h = arguments.VectorFunction("h", C)
h

h(x, y, z)

In [24]:
H = h.doit()
H

(H_r(r, theta, z))*C.b_r + (H_\theta(r, theta, z))*C.b_theta + (H_z(r, theta, z))*C.b_z

In [25]:
D = d.doit()
D

((-r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*Derivative(L_k^{(1)}(z), z) + phi_i^{(2)}(r)*L_k^{(2)}(z)*Derivative(psi_j^{(2)}(theta), theta))/r)*C.b_r + ((phi_i^{(0)}(r)*psi_j^{(0)}(theta)*Derivative(L_k^{(0)}(z), z) - psi_j^{(2)}(theta)*L_k^{(2)}(z)*Derivative(phi_i^{(2)}(r), r))/r)*C.b_theta + ((r**2*psi_j^{(1)}(theta)*L_k^{(1)}(z)*Derivative(phi_i^{(1)}(r), r) + 2*r*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z) - phi_i^{(0)}(r)*L_k^{(0)}(z)*Derivative(psi_j^{(0)}(theta), theta))/r)*C.b_z

In [26]:
R = C.to_cartesian(D)

In [27]:
R

(-(phi_i^{(0)}(r)*psi_j^{(0)}(theta)*Derivative(L_k^{(0)}(z), z) - psi_j^{(2)}(theta)*L_k^{(2)}(z)*Derivative(phi_i^{(2)}(r), r))*sin(theta) + (-r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*Derivative(L_k^{(1)}(z), z) + phi_i^{(2)}(r)*L_k^{(2)}(z)*Derivative(psi_j^{(2)}(theta), theta))*cos(theta)/r)*R.i + ((phi_i^{(0)}(r)*psi_j^{(0)}(theta)*Derivative(L_k^{(0)}(z), z) - psi_j^{(2)}(theta)*L_k^{(2)}(z)*Derivative(phi_i^{(2)}(r), r))*cos(theta) + (-r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*Derivative(L_k^{(1)}(z), z) + phi_i^{(2)}(r)*L_k^{(2)}(z)*Derivative(psi_j^{(2)}(theta), theta))*sin(theta)/r)*R.j + ((r**2*psi_j^{(1)}(theta)*L_k^{(1)}(z)*Derivative(phi_i^{(1)}(r), r) + 2*r*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z) - phi_i^{(0)}(r)*L_k^{(0)}(z)*Derivative(psi_j^{(0)}(theta), theta))/r)*R.k

In [28]:
u = TrialFunction(V, name="u")
uv = Dot(u, v)
srepr(uv)

"Dot(TrialFunction(x, y, z, Symbol('V')), TestFunction(x, y, z, Symbol('V')))"

In [29]:
UV = uv.doit()
UV

r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z)*phi_l^{(1)}(r)*psi_m^{(1)}(theta)*L_n^{(1)}(z) + phi_i^{(0)}(r)*psi_j^{(0)}(theta)*L_k^{(0)}(z)*phi_l^{(0)}(r)*psi_m^{(0)}(theta)*L_n^{(0)}(z) + phi_i^{(2)}(r)*psi_j^{(2)}(theta)*L_k^{(2)}(z)*phi_l^{(2)}(r)*psi_m^{(2)}(theta)*L_n^{(2)}(z)

In [30]:
UV.args

(phi_i^{(0)}(r)*psi_j^{(0)}(theta)*L_k^{(0)}(z)*phi_l^{(0)}(r)*psi_m^{(0)}(theta)*L_n^{(0)}(z),
 phi_i^{(2)}(r)*psi_j^{(2)}(theta)*L_k^{(2)}(z)*phi_l^{(2)}(r)*psi_m^{(2)}(theta)*L_n^{(2)}(z),
 r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z)*phi_l^{(1)}(r)*psi_m^{(1)}(theta)*L_n^{(1)}(z))

In [31]:
a = sp.separatevars(UV.args[2], dict=True, symbols=C._base_scalars)

In [32]:
a

{r: r**2*phi_i^{(1)}(r)*phi_l^{(1)}(r),
 theta: psi_j^{(1)}(theta)*psi_m^{(1)}(theta),
 z: L_k^{(1)}(z)*L_n^{(1)}(z),
 'coeff': 1}

In [33]:
for r in C._base_scalars:
    for j in a[r].args:
        try:
            print(j.local_index)
        except AttributeError:
            pass

0
0
1
1
2
2


In [34]:
v.doit()

(phi_i^{(0)}(r)*psi_j^{(0)}(theta)*L_k^{(0)}(z))*C.b_r + (phi_i^{(1)}(r)*psi_j^{(1)}(theta)*L_k^{(1)}(z))*C.b_theta + (phi_i^{(2)}(r)*psi_j^{(2)}(theta)*L_k^{(2)}(z))*C.b_z

In [35]:
for p in a[C._base_scalars[0]].args:
    print(p, isinstance(p, arguments.BasisFunction), p.doit())

r**2 False r**2
phi_i^{(1)}(r) True phi_i^{(1)}(r)
phi_l^{(1)}(r) True phi_l^{(1)}(r)


In [36]:
k = a[C._base_scalars[0]].args[0]

In [37]:
C._map_base_scalar_to_symbol

{r: r, theta: theta, z: z}